In [2]:
import os
import math
import argparse
import random

import torch.optim.lr_scheduler as lr_scheduler
import torch
import torch.optim as optim
import json
import glob
from torch.utils.tensorboard import SummaryWriter
from torchvision import transforms
from PIL import Image
from torchvision import transforms, datasets
device = torch.device("cuda")
print(device)
tb_writer = SummaryWriter()
if os.path.exists("./save_weights") is False:
    os.makedirs("./save_weights")
batch_size = 16

nw = min([os.cpu_count(), batch_size if batch_size > 1 else 0, 8])  # number of workers
print('Using {} dataloader workers every process'.format(nw))

cuda
Using 8 dataloader workers every process


In [ ]:
data_root = os.path.abspath(os.path.join(os.getcwd(), "../.."))
assert os.path.exists(data_root),"file: {} does not exist".format(data_root)

train_dir = os.path.join(data_root, "train")
val_dir = os.path.join(data_root, "val")

flower_class = [cla for cla in os.listdir(train_dir) if os.path.isdir(os.path.join(train_dir, cla))]
class_dict = dict((value, index) for index, value in enumerate(flower_class))
class_indices = dict((k, v) for v,k in enumerate(flower_class))
json_str = json.dumps(dict((val, key) for key, val in class_indices.items()), indent=4)
with open('class.json', 'w') as json_file:
    json_file.write(json_str)

train_img_path = glob.glob(train_dir + "/*/*.jpg")
random.shuffle(train_img_path)
train_label_list = [class_dict[path.split(os.path.sep)[-2]] for path in train_img_path]
train_num = len(train_img_path)

val_img_path = glob.glob(val_dir + "/*/*.jpg" )
random.shuffle(val_img_path)
val_label_list = [class_dict[path.split(os.path.sep)[-2]] for path in val_img_path]
val_num = len(val_img_path)

In [ ]:
img_size = {"B0": 224,
            "B1": 240,
           "B2": 260,
            "B3": 300,
            "B4": 380,
            "B5": 456,
            "B6": 528,
            "B7": 600}
num_model = "B0"



data_transform = {
    "train": transforms.Compose([transforms.RandomResizedCrop(img_size[num_model]),
                                 transforms.RandomHorizontalFlip(),
                                 transforms.ToTensor(),
                                 transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])]),
    "val": transforms.Compose([transforms.Resize(img_size[num_model]),
                               transforms.CenterCrop(img_size[num_model]),
                               transforms.ToTensor(),
                               transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
}

train_dataset = datasets.ImageFolder(train_dir, transform=data_transform['train'])
val_dataset = datasets.ImageFolder(val_dir, transform=data_transform['val'])

train_num = len(train_dataset)
validate_num = len(validate_dataset)

train_loader = torch.utils.data.DataLoader(train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True,
                                           pin_memory=True,
                                           num_workers=nw,
                                           collate_fn=train_dataset.coll
                                           )